In [ ]:
import pandas as pd
from typing import *
import numpy as np
from itertools import chain
import random
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from models.transformer import Transformer, TransformerSeparated

from dataset.splitter import split
from trainer import Trainer
from models.base_model import BaseModel
import sys

: 

In [24]:
import torch

# Verificar si la GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Crear un tensor y moverlo a la GPU (si disponible)
tensor = torch.randn(3, 3)
tensor = tensor.to(device)  # Mover tensor a la GPU si CUDA está disponible
print(device)
torch.__version__


cuda


'2.7.0+cu118'

In [25]:
from datasets import load_dataset

ds = load_dataset("Bena345/cdc-diabetes-health-indicators")

df = ds['train'].to_pandas()
#print(df.isnull().sum())
df['Diabetes_binary'].unique()
df

,ID,BMI,PhysHlth,Age,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,...,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,DiffWalk,Sex,Education,Income,Diabetes_binary
0,114414,29.0,0.0,65 to 69,0,1,1,0,0,0,...,0,1,1,Poor,0,0,0,6,7,Non-Diabetic
1,168896,32.0,0.0,80 or older,1,1,1,0,0,0,...,0,1,0,Poor,0,0,0,Some College Degree,4,Non-Diabetic
2,68354,25.0,5.0,65 to 69,1,0,1,1,0,0,...,0,1,0,Fair,0,0,1,6,2,Non-Diabetic
3,121194,24.0,0.0,80 or older,1,0,1,0,0,0,...,0,1,0,Very Good,0,1,0,Advanced Degree,5,Non-Diabetic
4,141150,31.0,0.0,25 to 29,0,0,1,0,0,1,...,0,1,0,Very Good,5,0,0,6,6,Diabetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194820,43401,28.0,28.0,50 to 54,1,1,1,0,0,0,...,0,1,0,Very Good,15,1,1,Advanced Degree,6,Diabetic
194821,233048,25.0,0.0,60 to 64,0,0,1,1,0,0,...,0,1,1,Fair,0,0,0,Advanced Degree,7,Non-Diabetic
194822,1241,33.0,1.0,65 to 69,0,1,1,1,0,1,...,0,1,0,Fair,2,0,0,6,7,Non-Diabetic
194823,248165,33.0,0.0,60 to 64,0,0,1,0,0,0,...,0,1,0,Fair,1,1,0,6,7,Non-Diabetic


In [ ]:
for col in df.columns:
    print((df[col].unique()), col, '\n')


[114414 168896  68354 ...   1241 248165 133109] ID 

[29. 32. 25. 24. 31. 20. 35. 28. 26. 23. 30. 33. 79. 43. 36. 27. 38. 22.
 44. 21. 45. 49. 39. 41. 73. 37. 42. 34. 52. 40. 58. 19. 17. 18. 51. 46.
 50. 56. 48. 86. 47. 15. 55. 92. 65. 76. 84. 53. 13. 87. 16. 62. 74. 89.
 64. 71. 61. 66. 59. 57. 83. 14. 63. 95. 81. 54. 75. 77. 60. 70. 82. 68.
 96. 12. 88. 72. 67. 78. 98. 90. 69. 91. 85.] BMI 

[ 0.  5.  2.  3. 10. 20.  4. 15. 25.  1. 14. 30.  7. 21. 23. 17.  6. 12.
  8. 28. 29. 27.  9. 26. 22. 13. 16. 18. 24. 11. 19.] PhysHlth 

['65 to 69' '80 or older' '25 to 29' '55 to 59' '70 to 74' '60 to 64'
 '75 to 79' '40 to 44' '35 to 39' '45 to 49' '18 to 24' '30 to 34'
 '50 to 54'] Age 

['Normal blood pressure' 'High blood pressure'] HighBP 

['High cholesterol' 'Low cholesterol'] HighChol 

['1' '0'] CholCheck 

['Non-smoker' 'Smoker'] Smoker 

['No stroke history' 'Had a stroke'] Stroke 

['No heart condition' 'Had heart disease'] HeartDiseaseorAttack 

['Physically active' 'Physically in

: 

In [27]:
dataset = df
binary_to_categorical = {
                "HighChol": {0: "Low cholesterol", 1: "High cholesterol"},
                "HvyAlcoholConsump": {0: "Not heavy drinker", 1: "Heavy alcohol use"},
                "Sex": {0: "Female", 1: "Male"},
                "MentHlth": {0: "Mentally healthy", 1: "Mental health issues"},
                "HeartDiseaseorAttack": {0: "No heart condition", 1: "Had heart disease"},
                "Stroke": {0: "No stroke history", 1: "Had a stroke"},
                "Smoker": {0: "Non-smoker", 1: "Smoker"},
                "HighBP": {0: "Normal blood pressure", 1: "High blood pressure"},
                "PhysActivity": {0: "Physically inactive", 1: "Physically active"},
            }
binary_to_categorical = {
    "HighChol": {"0": "Low cholesterol", "1": "High cholesterol"},  # 0: "Low cholesterol", 1: "High cholesterol"
    "HvyAlcoholConsump": {"0": "Not heavy drinker", "1": "Heavy alcohol use"},
    "Sex": {"0": "Female", "1": "Male"},
    "MentHlth": {"0": "Mentally healthy", "1": "Mental health issues"},
    "HeartDiseaseorAttack": {"0": "No heart condition", "1": "Had heart disease"},
    "Stroke": {"0": "No stroke history", "1": "Had a stroke"},
    "Smoker": {"0": "Non-smoker", "1": "Smoker"},
    "HighBP": {"0": "Normal blood pressure", "1": "High blood pressure"},
    "PhysActivity": {"0": "Physically inactive", "1": "Physically active"},
}

        # Aplicar mapping a cada columna
for col, mapping in binary_to_categorical.items():
    if col in df.columns:
        if set(df[col].dropna().unique()).issubset(mapping.keys()):
            df[col] = df[col].map(mapping)
        else:
            print(f"Avisa: La columna {col} té valors fora del mapping. No s'ha convertit.")


Avisa: La columna MentHlth té valors fora del mapping. No s'ha convertit.


In [28]:
df

,ID,BMI,PhysHlth,Age,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,...,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,DiffWalk,Sex,Education,Income,Diabetes_binary
0,114414,29.0,0.0,65 to 69,Normal blood pressure,High cholesterol,1,Non-smoker,No stroke history,No heart condition,...,Not heavy drinker,1,1,Poor,0,0,Female,6,7,Non-Diabetic
1,168896,32.0,0.0,80 or older,High blood pressure,High cholesterol,1,Non-smoker,No stroke history,No heart condition,...,Not heavy drinker,1,0,Poor,0,0,Female,Some College Degree,4,Non-Diabetic
2,68354,25.0,5.0,65 to 69,High blood pressure,Low cholesterol,1,Smoker,No stroke history,No heart condition,...,Not heavy drinker,1,0,Fair,0,0,Male,6,2,Non-Diabetic
3,121194,24.0,0.0,80 or older,High blood pressure,Low cholesterol,1,Non-smoker,No stroke history,No heart condition,...,Not heavy drinker,1,0,Very Good,0,1,Female,Advanced Degree,5,Non-Diabetic
4,141150,31.0,0.0,25 to 29,Normal blood pressure,Low cholesterol,1,Non-smoker,No stroke history,Had heart disease,...,Not heavy drinker,1,0,Very Good,5,0,Female,6,6,Diabetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194820,43401,28.0,28.0,50 to 54,High blood pressure,High cholesterol,1,Non-smoker,No stroke history,No heart condition,...,Not heavy drinker,1,0,Very Good,15,1,Male,Advanced Degree,6,Diabetic
194821,233048,25.0,0.0,60 to 64,Normal blood pressure,Low cholesterol,1,Smoker,No stroke history,No heart condition,...,Not heavy drinker,1,1,Fair,0,0,Female,Advanced Degree,7,Non-Diabetic
194822,1241,33.0,1.0,65 to 69,Normal blood pressure,High cholesterol,1,Smoker,No stroke history,Had heart disease,...,Not heavy drinker,1,0,Fair,2,0,Female,6,7,Non-Diabetic
194823,248165,33.0,0.0,60 to 64,Normal blood pressure,Low cholesterol,1,Non-smoker,No stroke history,No heart condition,...,Not heavy drinker,1,0,Fair,1,1,Female,6,7,Non-Diabetic


In [4]:
"birthdate"  or "Birthdate" in df.columns


'birthdate'

In [5]:
from dataset.radiolung import BaseDataset

data = BaseDataset( path_df = df, label_key="Diabetes_binary")

patient_id
BMI
PhysHlth
Age
HighBP
HighChol
CholCheck
Smoker
Stroke
HeartDiseaseorAttack
PhysActivity
Fruits
Veggies
HvyAlcoholConsump
AnyHealthcare
NoDocbcCost
GenHlth
MentHlth
DiffWalk
Sex
Education
Income
Diabetes_binary


In [ ]:




model = Transformer(dataset=data, 
                    feature_keys=[ "Income", "Education", "BMI", "Age", "HighBP", "HighChol", "CholCheck",
                                "Stroke", "Smoker", "Veggies", "PhysHlth",  "HvyAlcoholConsump",
                                "AnyHealthcare", "GenHlth", "MentHlth", "Sex",
                                "Fruits", "PhysActivity"],
                    label_key="Diabetes_binary", # imagin que es lo que s'ha de predir
                    mode="binary", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=256, merged= True )

In [7]:
from torch.utils.data import Sampler
import numpy as np

class FixedRatioBatchSampler(Sampler):
    def __init__(self, labels, batch_size=100, pos_ratio=0.14):
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.pos_ratio = pos_ratio
        self.n_pos = int(round(batch_size * pos_ratio))
        self.n_neg = batch_size - self.n_pos

        self.pos_indices = np.where(self.labels == 1)[0]
        self.neg_indices = np.where(self.labels == 0)[0]

        self.num_batches = min(
            len(self.pos_indices) // self.n_pos,
            len(self.neg_indices) // self.n_neg
        )

    def __iter__(self):
        pos = np.random.permutation(self.pos_indices)
        neg = np.random.permutation(self.neg_indices)

        for i in range(self.num_batches):
            pos_batch = pos[i*self.n_pos:(i+1)*self.n_pos]
            neg_batch = neg[i*self.n_neg:(i+1)*self.n_neg]
            batch = np.concatenate([pos_batch, neg_batch])
            np.random.shuffle(batch)
            yield batch.tolist()

    def __len__(self):
        return self.num_batches


In [1]:
tupla = (0.7,0.2,0)
tupla[1]

0.2

In [8]:

def collate_fn_dict(batch:List):
    return {key: [d[key] for d in batch] for key in batch[0].keys()}
train_ds, val_ds, test_ds,  train_labels, val_labels, test_labels = split(data, (0.7, 0.3,0))
#train_loader = DataLoader(data, batch_size=128, shuffle=True,  collate_fn=collate_fn_dict,)
#val_loader = DataLoader(val_ds, batch_size=128, shuffle=True,  collate_fn=collate_fn_dict,)
labels = df['Diabetes_binary']
labels = [ x[0] for x in labels]



In [10]:

batch_sampler_val = FixedRatioBatchSampler(
    labels=val_labels,
    batch_size=128,
    pos_ratio=0.14
)

val_loader = DataLoader(
    dataset=val_ds,
    batch_sampler=batch_sampler_val, num_workers=0,  collate_fn=collate_fn_dict,)


batch_sampler_train = FixedRatioBatchSampler(
    labels=val_labels,
    batch_size=128,
    pos_ratio=0.14
)

train_loader = DataLoader(
    dataset=train_ds,
    batch_sampler=batch_sampler_train, num_workers=0,  collate_fn=collate_fn_dict,)

In [12]:

#test_loader = DataLoader(test_ds, batch_size=16, shuffle=False,  collate_fn=collate_fn_dict,)

optimizer_params = {"lr": 1e-3}

df = data.get_df()

labels = df['Diabetes_binary']
labels = [ x[0] for x in labels]


trainer = Trainer(model=model, exp_name= 'dataset2_sampler_ratios_augment_particio' )
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=100, optimizer_params=optimizer_params , labels = labels, label_key="Diabetes_binary")

wandb: Currently logged in as: xiscacompany to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Classes weights: [0.1393353 0.8606647]


Epoch 0 / 100: 100%|██████████| 446/446 [00:24<00:00, 18.13it/s]


Classes weights: [0.1393353 0.8606647]


Epoch 1 / 100: 100%|██████████| 446/446 [00:27<00:00, 16.12it/s]


...

Epoch 99 / 100: 100%|██████████| 446/446 [00:27<00:00, 16.05it/s]


Classes weights: [0.1393353 0.8606647]


Evaluation: 100%|██████████| 446/446 [00:19<00:00, 22.93it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▇▇▇▇▇▇███
loss_train,███▇▇▆▆▆▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁
loss_val,▁▂▂▁▁▁▂▃▁▂▃▂▂▂▃▃▂▄▂▃▄▃▅▄▅▆▃▃▅▅▄▆▅▆▅▇▆▅▇█
recall0,▂▁▁▂▂▂▃▄▄▄▅▅▄▄▅▅▅▅▆▅▅▆▅▆▇▅▆▆▆▇▇▇▇▇▇▇▇▇██
recall0_val,▅▃█▆▆▅▄▅▃▃▃▃▄▄▄▃▃▃▂▄▃▄▁▂▄▂▂▃▂▂▄▃▃▃▃▄▃▄▃▄
recall1,▁▁▂▂▃▂▃▃▃▃▃▄▃▄▄▄▄▅▄▅▅▅▆▆▆▅▆▆▆▆▇▆▆▇▇▇▇▇▇█
recall1_val,▃▃▁▆▄▃▅▆▄██▅▅▅▆▇▇▅▆▆▃█▅▄▆▄▇▆▆▅▄▄▅▅▃▄▄▄▄▃
epoch,99
loss_train,0.45653
loss_val,1.18822
recall0,0.73335


{'loss': tensor(0.4985, device='cuda:0', grad_fn=<NllLossBackward0>),
 'acc': tensor(0.7344, device='cuda:0'),
 'y_prob': tensor([[9.2667e-01, 7.3328e-02],
         [2.5548e-01, 7.4452e-01],
         [9.9995e-01, 5.3221e-05],
         [8.6062e-01, 1.3938e-01],
         [3.1033e-01, 6.8967e-01],
         [1.8388e-01, 8.1612e-01],
         [9.9965e-01, 3.4973e-04],
         [2.5726e-01, 7.4274e-01],
         [9.9669e-01, 3.3147e-03],
         [2.1538e-01, 7.8462e-01],
         [2.5577e-01, 7.4423e-01],
         [8.9392e-01, 1.0608e-01],
         [9.8781e-01, 1.2195e-02],
         [2.0890e-01, 7.9110e-01],
         [6.5206e-01, 3.4794e-01],
         [1.1470e-01, 8.8530e-01],
         [9.8329e-01, 1.6709e-02],
         [9.6741e-01, 3.2593e-02],
         [9.8797e-01, 1.2027e-02],
         [9.9996e-01, 3.8477e-05],
         [6.0847e-01, 3.9153e-01],
         [2.2811e-01, 7.7189e-01],
         [2.3524e-01, 7.6476e-01],
         [6.8142e-01, 3.1858e-01],
         [7.4788e-01, 2.5212e-01],
    

In [ ]:



model = Transformer(dataset=data, 
                    feature_keys=[ "Income", "Education", "BMI", "Age", "HighBP", "HighChol", "CholCheck",
                                "Stroke", "Smoker", "Veggies", "PhysHlth", "CholCheck", "HvyAlcoholConsump",
                                "AnyHealthcare", "GenHlth", "MentHlth", "Sex",
                                "Fruits", "PhysActivity"],
                    label_key="Diabetes_binary", # imagin que es lo que s'ha de predir
                    mode="binary", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=64, merged= False )




def collate_fn_dict(batch:List):
    return {key: [d[key] for d in batch] for key in batch[0].keys()}
train_ds, val_ds, test_ds = split(data, (0.8, 0.2,0))
train_loader = DataLoader(data, batch_size=256, shuffle=True,  collate_fn=collate_fn_dict,)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=True,  collate_fn=collate_fn_dict,)
optimizer_params = {"lr": 1e-3}

df = data.get_df()

labels = df['Diabetes_binary']
labels = [ x[0] for x in labels]
 

trainer = Trainer(model=model, exp_name= 'dataset2_train_test' )
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=100, optimizer_params=optimizer_params , labels = labels, label_key="Diabetes_binary")

Classes weights: [0.1393353 0.8606647]


Epoch 0 / 100: 100%|██████████| 762/762 [00:55<00:00, 13.83it/s]


Classes weights: [0.1393353 0.8606647]


Epoch 1 / 100: 100%|██████████| 762/762 [01:04<00:00, 11.86it/s]


Classes weights: [0.1393353 0.8606647]


...

Epoch 99 / 100: 100%|██████████| 762/762 [00:57<00:00, 13.22it/s]


Classes weights: [0.1393353 0.8606647]


Evaluation: 100%|██████████| 115/115 [00:07<00:00, 16.42it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
loss_train,██▆▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁
loss_val,▂▁▂▁▂▂▂▂▂▄▃▂▃▂▄▃▃▂▃▃▄▅▃▃▅▄▄▇▅▅▄▇▃▅▆▅▆█▄▆
recall0,▁▁▆▆▆▇▆▆▇▆▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇█▇▇█▇█▇▇███▇██
recall0_val,▇▇█▇█▆▅▅▅▆▅▄▅▅▄▄▄▆▅▅▅▄▅▅▅▄▃▂▃▄▅▄▄▄▃▁▁▂▄▃
recall1,▁▂▂▄▄▄▅▅▄▄▅▆▆▅▅▅▆▆▆▆▆▇▇▆▇▆▆▇▇▇▇▇█▇▇██▇▇█
recall1_val,▄▁▁▄▄▁▄▅▅▃▆▅▆▇▅▅▅▇▅▆▆▆▇▆▇▆▅██▇▄▃▅▇█▇▇▃██
epoch,99
loss_train,0.5002
loss_val,0.69355
recall0,0.7053


{'loss': tensor(0.5456, device='cuda:0', grad_fn=<NllLossBackward0>),
 'acc': tensor(0.6667, device='cuda:0'),
 'y_prob': tensor([[0.3004, 0.6996],
         [0.7449, 0.2551],
         [0.9979, 0.0021],
         [0.4679, 0.5321],
         [0.9860, 0.0140],
         [0.9008, 0.0992],
         [0.6589, 0.3411],
         [0.6000, 0.4000],
         [0.2889, 0.7111]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 'y_true': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1], device='cuda:0'),
 'logit': tensor([[-0.1900,  0.6553],
         [ 0.2094, -0.8623],
         [ 3.9596, -2.1944],
         [ 0.2590,  0.3877],
         [-0.3340, -4.5894],
         [-0.7977, -3.0038],
         [ 0.1238, -0.5347],
         [ 0.2299, -0.1756],
         [-0.7199,  0.1808]], device='cuda:0', grad_fn=<AddmmBackward0>),
 'classification_report': {'0': {'precision': 0.8333333333333334,
   'recall': 0.7142857142857143,
   'f1-score': 0.7692307692307693,
   'support': 7.0},
  '1': {'precision': 0.3333333333333333,
   'recall': 0

In [ ]:
 mapping = {0: "Low Chol", 1: "High Chol"}
self.df[self.label_key ] = self.df["HighChol" ].map(mapping)